# Daily Fantasy Basketball Predictions

## Get the player averages, info and salaries from draftkings, an example csv is in the repository
## Take the opponent and find the average points allowed by that team (based on basketball monster website)
## Adjust the average score of the player by the ratio of the avg points allowed by the opponent/ the average points allowed by all of the teams
## Multiplying by this ratio will hopefully somewhat adjust for competition of the night



$Player's Projected Points = Player's Avg Points * \frac{Avg Points Allowed By Opponent}{Avg Points Allowed By All Teams}$

### Install necessary packages:

In [16]:
# Import for data analysis:
import numpy as np
import pandas as pd
# Import for web scraping:
import requests
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import urllib.request
import csv

### Get players from todays games, figure out their opponent :

In [17]:
# Get info from CSV downloaded from DraftKings... Better way to get CSV?  Download automatically somehow?
todays_players_df = pd.read_csv('DKSalaries.csv')

# Create new columns in dataframe for the opponent and the new projection updated based on opponent
todays_players_df["Opponent"] = "NA"
todays_players_df["Proj"] = todays_players_df["AvgPointsPerGame"]
#print(todays_players_df)

# List of all teams with abreviations that are 2 letters long, need this to find opponent team
two_letter_teams_list = ["GS", "NY", "NO","SA"]

# Loop through all players who are playing today
for playerstats in todays_players_df.iterrows():
    ############# Find Players Opponent #############
    gameInfo = playerstats[1]["Game Info"]
    playersTeam = playerstats[1]["TeamAbbrev"]
    # Ex:  want to get the team Harden is playing from game info string "HOU@NY 11/16/2019 08:00PM ET" 
    try:
        indexVs = gameInfo.index("@")
        # If the team before @ is not the players team, it is the opponent
        if (gameInfo[0:indexVs] != playersTeam):
            opposingTeam = gameInfo[0:indexVs]
        # Else if the team listed after the @ is the opponent and the team has 2 letters
        elif (gameInfo[indexVs:indexVs+2] in two_letter_teams_list):
            opposingTeam = gameInfo[indexVs+1:indexVs+3]
        else:
            opposingTeam = gameInfo[indexVs+1:indexVs+4]
    except:
        opposingTeam = "NA"
    
    # Store the opponents in another row in the playerstats dataframe
    # playerstats[0] is the index of the player in the dataframe
    todays_players_df.loc[playerstats[0],"Opponent"] = opposingTeam
    
print(todays_players_df)

    Position                          Name + ID                    Name  \
0      PG/SG            James Harden (13772382)            James Harden   
1      SF/PF   Giannis Antetokounmpo (13772386)   Giannis Antetokounmpo   
2      PG/SG             Luka Doncic (13772390)             Luka Doncic   
3          C      Karl-Anthony Towns (13772394)      Karl-Anthony Towns   
4      SF/PF           Kawhi Leonard (13772396)           Kawhi Leonard   
5         PG              Trae Young (13772400)              Trae Young   
6       PF/C           Pascal Siakam (13772403)           Pascal Siakam   
7         PG          Damian Lillard (13772407)          Damian Lillard   
8      PG/SG            Jrue Holiday (13772410)            Jrue Holiday   
9       PF/C        Domantas Sabonis (13772414)        Domantas Sabonis   
10     SG/SF          Andrew Wiggins (13772418)          Andrew Wiggins   
11        PG       Russell Westbrook (13772423)       Russell Westbrook   
12     SF/PF             

### Get the avg points allowed by each team, for now stick with total, not per position

In [18]:
# Following BeautifulSoup beginners tutorial
URL = 'https://basketballmonster.com/dfsdvp.aspx'
response = requests.get(URL)
soup = BeautifulSoup(response.content, 'html.parser')
labels = []
data = []
rows=list()
    
table = soup.find_all('table')[0] 

# df is a list of all of the found data frames. len = 1
df = pd.read_html(str(table))
# So the dataframe we want to use is the only one found at index 0
df_opponentPtsAllowed = df[0]

# Take out "vs " in the "Team" column 
for row in df2.index:
    df_opponentPtsAllowed.loc[row,"Team"] = df2.loc[row,"Team"][0][3:]

# Make the indexing column be the Team
# Note:  For some reason when just using the "Team" collumn directly, the indecies become tuples
df_opponentPtsAllowed["TeamStr"] = df2["Team"]
df_opponentPtsAllowed = df_opponentPtsAllowed.set_index("TeamStr")

print(df_opponentPtsAllowed)

    
    



                      Team     g    All    PG    SG    SF    PF     C
        Unnamed: 0_level_1   Avg  224.1  46.2  41.7  40.3  44.3  51.8
TeamStr                                                              
NaN                 vs ATL  67.0  242.3  47.9  44.8  45.8  47.8  56.0
NaN                 vs CLE  65.0  238.3  50.4  44.8  43.8  45.8  53.4
NaN                 vs WAS  64.0  234.1  49.4  43.0  42.5  45.3  54.4
NaN                 vs MIN  64.0  233.1  48.1  44.4  42.0  46.5  52.8
NaN                 vs GSW  65.0  233.0  49.3  43.9  42.4  44.6  53.0
NaN                 vs NOR  64.0  232.2  50.3  42.8  41.2  46.2  52.9
NaN                 vs POR  66.0  232.0  48.4  42.0  40.8  47.1  54.3
NaN                 vs HOU  64.0  231.0  46.9  43.7  41.6  47.5  51.9
NaN                 vs MEM  65.0  230.2  46.7  43.9  41.7  45.1  52.8
NaN                 vs CHA  65.0  228.4  43.9  42.0  40.4  46.8  55.8
NaN                 vs PHO  65.0  227.0  46.7  42.4  40.8  44.2  52.9
NaN                 

In [19]:
print(df_opponentPtsAllowed.loc["ATL","All"][0])

nan


### Adjust the players projected score based on their opponents avg points allowed, for now stick with total, not per position

In [20]:
avgAllTeams = df2.columns.levels[1][0]
print(avgAllTeams)

# Loop through all players who are playing today
for i, playerstats in todays_players_df.iterrows():
    print("**************************************************")
    print("Original Info: ")
    print(playerstats)
    # The Two letter teams cause issues again, better solution for this?
    opponent = playerstats["Opponent"].strip()
    if (opponent == "SA"):
        opponent = "SAS"
    elif(opponent == "GS"):
        opponent = "SAS"
    elif(opponent == "NO"):
        opponent = "NOR"
    elif(opponent == "NY"):
        opponent = "NYK"
        
    print("Points allowed by opponent/avg of all teams: "+str(df2.loc[opponent,"All"][0]/ float(avgAllTeams)))
    print("*************************")
    todays_players_df.loc[i,"Proj"] = todays_players_df.loc[i,"AvgPointsPerGame"] * (df_opponentPtsAllowed.loc[opponent,"All"][0]/ float(avgAllTeams))
    
    # print(playerstats)
    print(todays_players_df.loc[i,"Proj"])
    

224.1
**************************************************
Original Info: 
Position                                    PG/SG
Name + ID                 James Harden (13772382)
Name                                 James Harden
ID                                       13772382
Roster Position                      PG/SG/G/UTIL
Salary                                      12200
Game Info           HOU@MIN 11/16/2019 08:00PM ET
TeamAbbrev                                    HOU
AvgPointsPerGame                            62.33
Opponent                                      MIN
Proj                                        62.33
Name: 0, dtype: object
Points allowed by opponent/avg of all teams: 1.0401606425702812
*************************
nan
**************************************************
Original Info: 
Position                                       SF/PF
Name + ID           Giannis Antetokounmpo (13772386)
Name                           Giannis Antetokounmpo
ID                                

nan
**************************************************
Original Info: 
Position                                   SF/PF
Name + ID                    Rudy Gay (13772600)
Name                                    Rudy Gay
ID                                      13772600
Roster Position                     SF/PF/F/UTIL
Salary                                      4800
Game Info           POR@SA 11/16/2019 08:30PM ET
TeamAbbrev                                    SA
AvgPointsPerGame                           24.21
Opponent                                     POR
Proj                                       24.21
Name: 59, dtype: object
Points allowed by opponent/avg of all teams: 1.035252119589469
*************************
nan
**************************************************
Original Info: 
Position                                    SF/PF
Name + ID                   OG Anunoby (13772596)
Name                                   OG Anunoby
ID                                       13772596
Roster

nan
**************************************************
Original Info: 
Position                                    PG/SG
Name + ID               Shabazz Napier (13772866)
Name                               Shabazz Napier
ID                                       13772866
Roster Position                      PG/SG/G/UTIL
Salary                                       3400
Game Info           HOU@MIN 11/16/2019 08:00PM ET
TeamAbbrev                                    MIN
AvgPointsPerGame                            21.29
Opponent                                      HOU
Proj                                        21.29
Name: 125, dtype: object
Points allowed by opponent/avg of all teams: 1.0307898259705488
*************************
nan
**************************************************
Original Info: 
Position                                   SG/SF
Name + ID              Damyean Dotson (13772874)
Name                              Damyean Dotson
ID                                      137728

nan
**************************************************
Original Info: 
Position                                    SG/SF
Name + ID                Charlie Brown (13773285)
Name                                Charlie Brown
ID                                       13773285
Roster Position                    SG/SF/F/G/UTIL
Salary                                       3000
Game Info           ATL@LAC 11/16/2019 10:30PM ET
TeamAbbrev                                    ATL
AvgPointsPerGame                                4
Opponent                                      LAC
Proj                                            4
Name: 189, dtype: object
Points allowed by opponent/avg of all teams: 0.9955377063810799
*************************
nan
**************************************************
Original Info: 
Position                                        C
Name + ID               Bruno Fernando (13773290)
Name                               Bruno Fernando
ID                                       13

In [21]:
print(todays_players_df)

    Position                          Name + ID                    Name  \
0      PG/SG            James Harden (13772382)            James Harden   
1      SF/PF   Giannis Antetokounmpo (13772386)   Giannis Antetokounmpo   
2      PG/SG             Luka Doncic (13772390)             Luka Doncic   
3          C      Karl-Anthony Towns (13772394)      Karl-Anthony Towns   
4      SF/PF           Kawhi Leonard (13772396)           Kawhi Leonard   
5         PG              Trae Young (13772400)              Trae Young   
6       PF/C           Pascal Siakam (13772403)           Pascal Siakam   
7         PG          Damian Lillard (13772407)          Damian Lillard   
8      PG/SG            Jrue Holiday (13772410)            Jrue Holiday   
9       PF/C        Domantas Sabonis (13772414)        Domantas Sabonis   
10     SG/SF          Andrew Wiggins (13772418)          Andrew Wiggins   
11        PG       Russell Westbrook (13772423)       Russell Westbrook   
12     SF/PF             